In [1]:
# Fuseki IO Development V
## Refinement

# Attach dir one above to handle IDE subdir
import sys, os
module_path = os.path.abspath(os.path.join('../src/'))
if module_path not in sys.path:
    sys.path.append(module_path)

print(sys.path)
    
import repository
import loader
from rdflib import URIRef, Graph, Namespace

#from models.DMEAR import DMEAR_vis 
from visualize_model import visualize, t2rdflibg, get_field

import vis_rdf

import gravis as gv

import pandas as pd
import html
import io
import pydotplus
from rdflib.tools.rdf2dot import rdf2dot
from rdflib import URIRef, Graph, Namespace, Literal
from IPython.display import display, Image, HTML
import difflib
from urllib.error import HTTPError
from itertools import product
import re
from datetime import datetime

start = datetime.now()
import text_utils

['/usr/lib/python310.zip', '/usr/lib/python3.10', '/usr/lib/python3.10/lib-dynload', '', '/home/tomk/environment/devpy/lib/python3.10/site-packages', '/home/tomk/Documents/Coding/gitHub/daglit/src', '/home/tomk/Documents/Coding/gitHub/datamodels/src']


IndentationError: expected an indented block after 'elif' statement on line 418 (serialization.py, line 421)

In [ ]:
store_type="jena"
#store_type="memory"
repo = repository.Repository(store_type=store_type)

In [ ]:
discourse_e=URIRef("http://www.tkltd.org/ontologies/1635226058024d1e81c1070ee48143a5")
discourse_e=URIRef("http://www.tkltd.org/ontologies/749f90d34efa47c1ac3bda900a29d636")


qr = repo.run_cached_query("get_discourse_posits_parms_discourse_iris.sparql", parameters=[discourse_e.n3()], native_rdflib=True)
pd.DataFrame(qr)
g=Graph()
for row in qr:
    g.add((row['s'], row['p'], row['o']))

In [ ]:
gjgf=vis_rdf.process_graph(g)

model_html = gv.d3(gjgf, 
      node_label_data_source='label',
      show_edge_label=True,
      edge_label_size_factor=0.7,
      edge_label_data_source='label',
      edge_curvature=0.25,
     links_force_strength=0.8, 
     links_force_distance=55, 
     use_collision_force=True, 
     collision_force_radius=10, 
    collision_force_strength=1.0,
     many_body_force_strength=-1300,
                   many_body_force_theta=1.61, 
                   use_many_body_force_min_distance=True,
                   many_body_force_min_distance=0.01,
                   use_many_body_force_max_distance=False,
                   many_body_force_max_distance=40000,
                   use_x_positioning_force=True,
                   x_positioning_force_strength=0.21,
                   use_y_positioning_force=True,
                   y_positioning_force_strength=0.21
     ).to_html_partial()

HTML(model_html)

In [ ]:
vis_rdf.get_literal_properties_set(g)

In [ ]:
def get_literals_to_table(graph, entity):
    lit_kvs = [(t[1][t[1].find("#")+1:], t[2].toPython()) for t in graph.triples((entity, None, None)) if isinstance(t[2], Literal)]
    table_html_template="""
<table xmlns="http://www.w3.org/1999/xhtml"><thead><tr><th>Property</th><th>Value</th></tr></thead>
<tbody>
{rows}
</tbody>
</table>
    """
    
    rows=[]
    row_html_template="""<tr><th>{k}</th><td>{v}</td></tr>"""
    for k,v in lit_kvs:
        
        rows.append(row_html_template.format(k=k,v=(v+v)))
    rows = "".join(rows)
    return table_html_template.format(rows=rows)

In [ ]:
literals_table = get_literals_to_table(g, URIRef("http://www.tkltd.org/ontologies/37bf07d15e264bdc8c3fb17369bef0c6"))

HTML(literals_table)

In [ ]:
from panel import Panel
import gradients

In [ ]:
def make_path(panel, title_style, panel_style):
    path1 = " ".join(panel._title_path_steps())
    path2 = " ".join(panel._panel_path_steps())
    #stroke:none; fill:url(#pink_white);
    #stroke:none; fill:url(#blue);
    svg_path1 = """<path style="{style}" d="{p}"> </path> """.format(p=path1, style=title_style)
    svg_path2 = """<path style="{style}" d="{p}"> </path> """.format(p=path2, style=panel_style)
    return svg_path1 + svg_path2

In [ ]:
def foreign_object(shape, content):
    fo = """<foreignObject x="0" y="0" width="400px" height="100%">
    <div width="80%">{content}</div>
    </foreignObject>
"""    
    return fo.format(content=content)


In [ ]:
x=0
y=0
scale = 1

p=Panel(loc= ((x*scale)+5,(y*scale)+5), size=(400, 200), corner_radius_ratio=7, title_panel_ratio=0.25)

panel_paths = make_path(p, "stroke:black; stroke-width:1; fill:url(#white);", "stroke:black; stroke-width:1; fill:lightblue;")

In [ ]:
g_defs = gradients.rainbow_gradient() + gradients.miami_gradient() + gradients.blue_gradient_td() + gradients.blue_gradient_bu() + gradients.pink_orange_gradient() + gradients.pink_white_gradient_td()+ gradients.pink_white_gradient_bu()

In [ ]:
svg_chunk = """<svg version="1.1" xmlns="http://www.w3.org/2000/svg"
xmlns:xlink="http://www.w3.org/1999/xlink" x="20" y="20"
width="500" height="500" viewBox="0 0 600 300">
{defs}
  <g id="Layer_1">
    <g>{contents}
    </g>
  </g>
  {extras}
  </svg>
    """

In [ ]:
HTML(svg_chunk.format(defs=g_defs, contents = panel_paths + foreign_object("x", literals_table) , extras=None))

In [ ]:


HTML(f"""<svg version="1.1" xmlns="http://www.w3.org/2000/svg"

    
xmlns:xlink="http://www.w3.org/1999/xlink" x="0" y="0"
    
width="900" height="600" 
viewBox="-1, -1, 200, 200">
<defs>
<linearGradient id="miami" x1="0%" y1="0%" x2="0%" y2="80%" spreadMethod="pad">
            <stop stop-color="#20F" offset="0"/>
            <stop stop-color="#60fff0" offset="0.25"/>
            <stop stop-color="#80ffff" offset="0.5"/>
            <stop stop-color="#fff324" offset="0.7667"/>
            <stop stop-color="#fb24b3" offset="1"/>
            </linearGradient>
</defs>
<path style="fill: #FFAA00;" d="{title_path}"/><path class="stroke_red" d="{panel_path}"/></svg>""")

In [3]:
import re

In [16]:
payload_regex=re.compile("#LINK\((.+)\)")
payload_strings=payload_regex.findall("#LINK(data_1, data-2,data_3")

In [24]:
def resolve_label(label, row):
    if not str(label).startswith("#"):
        return row.get(label)
    elif str(label).startswith("#LINK"):
        # Perform LINK function to join input labels
        payload_regex=re.compile("(?:#LINK)\((.+)\)")
        payload_strings=payload_regex.findall(label)
        if len(payload_strings)==1:
            labels=[s.strip() for s in payload_strings[0].split(",")]
            print(labels)
            return ".".join([str(row.get(l,"_")) for l in labels])


In [25]:
row_d = {"data_1":1, "data-2" : 2, "data_3" : 3}
resolve_label("#LINK(data_1, data-2)", row_d)

['data_1', 'data-2']


'1.2'